In [3]:
import ee
import os
import geemap.core as geemap

if 'google.colab' in str(get_ipython()):
    from google.colab import userdata
    EE_PROJECT_ID = userdata.get('EE_PROJECT_ID') 
else:
    from dotenv import load_dotenv
    load_dotenv()  # take environment variables
    EE_PROJECT_ID = os.getenv('EE_PROJECT_ID')

# Set up GEE API
ee.Authenticate()
ee.Initialize(project=EE_PROJECT_ID) #<- Remember to change this to your own project's name!

In [4]:
# A Landsat 8 surface reflectance image with SWIR1, NIR, and green bands.
img = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_044034_20210508').select(
    ['SR_B6', 'SR_B5', 'SR_B3']
)

# Santa Cruz Mountains ecoregion geometry.
geom = (
    ee.FeatureCollection('EPA/Ecoregions/2013/L4')
    .filter('us_l4name == "Santa Cruz Mountains"')
    .geometry()
)

# Display layers on the map.
m = geemap.Map()
m.set_center(-122.08, 37.22, 9)
m.add_layer(img, {'min': 10000, 'max': 20000}, 'Landsat image')
m.add_layer(geom, {'color': 'white'}, 'Santa Cruz Mountains ecoregion')
display(m)

Map(center=[37.22, -122.08], controls=(WidgetControl(options=['position', 'transparent_bg'], position='toprigh…

In [5]:
# Calculate median band values within Santa Cruz Mountains ecoregion. It is
# good practice to explicitly define "scale" (or "crsTransform") and "crs"
# parameters of the analysis to avoid unexpected results from undesired
# defaults when e.g. reducing a composite image.
stats = img.reduceRegion(
    reducer=ee.Reducer.median(),
    geometry=geom,
    scale=30,  # meters
    crs='EPSG:3310',  # California Albers projection
)

# A dictionary is returned keys are band names, values are the statistic.
display('Median band values, Santa Cruz Mountains ecoregion', stats)

'Median band values, Santa Cruz Mountains ecoregion'

In [6]:
# You can combine reducers to calculate e.g. mean and standard deviation
# simultaneously. The output dictionary keys are the concatenation of the band
# names and statistic names, separated by an underscore.
reducer = ee.Reducer.mean().combine(
    reducer2=ee.Reducer.stdDev(), sharedInputs=True
)
multi_stats = img.reduceRegion(
    reducer=reducer,
    geometry=geom,
    scale=30,
    crs='EPSG:3310',
)
display('Mean & SD band values, Santa Cruz Mountains ecoregion', multi_stats)

'Mean & SD band values, Santa Cruz Mountains ecoregion'

In [8]:
percentiles

In [10]:
centered = False
name = 'Landsat image (1% linear stretch)'
pb_99 = ['SR_B3_p99', 'SR_B5_p99', 'SR_B6_p99']
pb_1 = ['SR_B3_p1', 'SR_B5_p1', 'SR_B6_p1']
percentiles = img.reduceRegion(ee.Reducer.percentile([1, 99]), maxPixels=1e11)
mx = percentiles.values(pb_99)
if centered:
    mn = ee.Array(mx).multiply(-1).toList()
else:
    mn = percentiles.values(pb_1)
m.addLayer(img, {'min': mn, 'max': mx}, name)

In [12]:
mn

In [13]:
mx